In [1]:
import os
os.environ['ISX'] = '0'

import numpy as np
import pandas as pd
from pathlib import Path

from dewan_calcium.helpers import IO

In [2]:
 ## animal and date must match old project exactly
animal = 'VGAT-39-Conc' 
date = '5-30-24'

fileHeader = animal + '-' + date + '-'

In [3]:
folder = ['ImagingAnalysis', 'AUROCData']
folder = Path(*folder)

onTimeAUROCSignificanceTable = IO.load_data_from_disk('onTimeAUROCSignificanceTable', fileHeader, folder)
latentAUROCSignificanceTable = IO.load_data_from_disk('latentAUROCSignificanceTable', fileHeader, folder)

folder = ['ImagingAnalysis', 'PreProcessedData']
folder = Path(*folder)

CellList = IO.load_data_from_disk('CellList', fileHeader, folder)
OdorData = IO.load_data_from_disk('OdorData', fileHeader, folder)

VGAT-39-Conc-5-30-24-onTimeAUROCSignificanceTable has loaded successfully!
VGAT-39-Conc-5-30-24-latentAUROCSignificanceTable has loaded successfully!
VGAT-39-Conc-5-30-24-CellList has loaded successfully!
VGAT-39-Conc-5-30-24-OdorData has loaded successfully!


In [7]:
new_significance_table = pd.DataFrame()

CellList = [f'C{num}' for num in CellList]
odors = np.unique(OdorData)
onTimeAUROCSignificanceTable = pd.DataFrame(onTimeAUROCSignificanceTable).astype(int).T
latentAUROCSignificanceTable = pd.DataFrame(latentAUROCSignificanceTable).astype(int).T
onTimeAUROCSignificanceTable.columns=CellList
latentAUROCSignificanceTable.columns=CellList
latentAUROCSignificanceTable[latentAUROCSignificanceTable==1] = 3
latentAUROCSignificanceTable[latentAUROCSignificanceTable==2] = 4

for cell in CellList:
    on_time_data = onTimeAUROCSignificanceTable[cell]
    latent_data = latentAUROCSignificanceTable[cell]
    
    zero_mask = on_time_data == 0
    
    significance_data = on_time_data.copy()
    
    significance_data.loc[zero_mask] = latent_data.loc[zero_mask]
    new_significance_table = pd.concat([new_significance_table,significance_data], axis=1)
    
new_significance_table.index=odors

significance_table_file_path = f'{fileHeader}SignificanceTable.xlsx'
new_significance_table.to_excel(significance_table_file_path)